In [1]:
#sc.stop()
#from pyspark import SparkContext 
#sc = SparkContext()


In [1]:
numPartitions = 10
baseDir = os.path.join('hdfs://ns372208.ip-91-121-199.eu/extract')
TwoMonthFilename = os.path.join(baseDir, 'extract_multimedia.tsv')

rawRatings = sc.textFile(TwoMonthFilename).repartition(numPartitions)

In [2]:
logs = rawRatings.map(lambda line: line.split('\t'))
logs = logs.filter(lambda x : len(x) == 8) # filter not completed rows


In [3]:
ProductList = (logs
               .map(lambda x:(x[6],1))
               .reduceByKey(lambda x,y : x+y, numPartitions=15)
               .keys()
               .collect()
               )


products_ints = dict()
ints_products = dict()

# iterate over all products labelled and mapping with a int rather 
i = 1
for product in ProductList:
    products_ints[product] = i
    ints_products[i] = product   
    i = i + 1


UserList =  (logs
             .map(lambda x:(x[0],1))
             .reduceByKey(lambda x,y : x+y, numPartitions=15)
             .keys()
             .collect()
            )

users_ints = dict()
ints_users = dict()

# iterate over all user hexa id and mapping with a int rather
i = 1
for user in UserList:
    users_ints[user] = i
    ints_users[i] = user   
    i = i + 1



In [4]:
logs = logs.map(lambda x:(users_ints[x[0]],x[1],x[2],x[3],x[4],x[5], products_ints[x[6]]))


In [5]:
logs.take(1)

[(1434403,
  u'5578b13d0cf28db65d6eed5d',
  u'Wed Jun 10 2015 23:50:42 GMT+0200 (CEST)',
  u'2500',
  u'10',
  u'access offer page',
  332)]

In [5]:
print logs.getNumPartitions()

10


# <span style="color:red">SEPARATION LOGS RASSEMBLES PAR TYPE</span> #

In [6]:

cartLogs = (logs 
            .filter(lambda x : x[5] in ('remove from cart','add to cart'))
           )
accessOfferLogs = (logs 
            .filter(lambda x : x[5] in ('access offer page'))
            .filter(lambda x : x[3] != '0')
           )
#oddLogs = (logs 
#            .filter(lambda x : x[5] in ('userActionLabel'))
#           )

print cartLogs.take(3)
print accessOfferLogs.take(3)
#print oddLogs.take(3)

[(596701, u'5578b1800cf28db65d6eee12', u'Wed Jun 10 2015 23:51:52 GMT+0200 (CEST)', u'0', u'3', u'add to cart', 1682), (1011603, u'561cb6d50cf258e83bc071a7', u'Tue Oct 13 2015 09:47:00 GMT+0200 (CEST)', u'0', u'6', u'remove from cart', 3543), (453372, u'561cb7630cf2fe9257a4c37a', u'Tue Oct 13 2015 09:49:10 GMT+0200 (CEST)', u'0', u'3', u'add to cart', 3543)]
[(688212, u'5578b16a0cf21c4200d0007d', u'Wed Jun 10 2015 23:50:53 GMT+0200 (CEST)', u'33514', u'10', u'access offer page', 1682), (1338275, u'564d8ab20cf2ed3701e96551', u'Thu Nov 19 2015 09:39:14 GMT+0100 (CET)', u'4699', u'10', u'access offer page', 3118), (559153, u'561cb6e10cf2fe9257a4c216', u'Tue Oct 13 2015 09:46:41 GMT+0200 (CEST)', u'42329', u'10', u'access offer page', 145)]


# <span style="color:red"> SUPRESSION DES ACCESS LOGS UNIQUES PAR USER </span>#
### Le collaborative filtering algorithm n'est pas capable de prédire des ratings à froid ###
#### (Préférer plutôt un modele baseline pour le cold start ou pour les users non identifiés) ####

In [7]:
onlyOneAccessOfferLogs = (accessOfferLogs
                                 .map(lambda x : ((x[0],x[6]),1))
                                 .reduceByKey(lambda x,y : x+y, numPartitions=15)
                                 .map(lambda ((x,y),z): (x,1))
                                 .reduceByKey(lambda x,y : x+y, numPartitions=22)
                                 .filter(lambda (x,y) : y==1)
                                 .map(lambda (x,y): (x,-1))
                                 )

# The list could be huge, so we broadcast it through the cluster 
# Other optimisation clue : Translate User Id into integers ..... 

#319493
#629127

In [8]:
print onlyOneAccessOfferLogs.getNumPartitions()

22


In [13]:
print accessOfferLogs.take(1)

[(361242, u'55a6d9d30cf21d087544dc06', u'Thu Jul 16 2015 00:08:19 GMT+0200 (CEST)', u'23732', u'10', u'access offer page', 2896)]


In [9]:
accessOfferLogs2 = (accessOfferLogs
                   .map(lambda x : (x[0],(x[1],x[2],x[3],x[4],x[5],x[6])))
                   .leftOuterJoin(onlyOneAccessOfferLogs, numPartitions=30)
                   .filter(lambda (x,y): y[1] != -1 )
                   .map(lambda (x,y) : (x,y[0][0],y[0][1],y[0][2],y[0][3],y[0][4],y[0][5]))
                   #.take(1)
                   )
#print accessOfferLogs.count()


In [ ]:
print accessOfferLogs2.take(5)

In [ ]:
#accessOfferLogs.saveAsTextFile('hdfs://ns372208.ip-91-121-199.eu/extract/extract08-08_10-08_onlyAccessOfferLogsFILTERED.tsv')

In [ ]:
print accessOfferLogs.filter(lambda x : x[3]== '0').count()
print accessOfferLogs.count()

## Gestion des "NaN values" ## 

In [ ]:
def returnMeANumber(string):
    if string.isdigit():
        return int(string)
    else:
        return 0
    
  

### Use cases ###
* #### 'access offer page' / duration : 0  ===> Pas d'action spécifiques donc interet très faible ####
* #### 'add to cart' / duration = 0 toujours ===> Interet fort ####


In [ ]:
############ TIME & DATE Labo ##########################################
import time
x = 'Sat Sep 19 2015 10:15:10 GMT+0200 (CEST)'
y = 'Sat Sep 19 2015 10:15:09 GMT+0200 (CEST)'
print x[:-16]
print time.strptime(x[:-16],"%a %b %d %Y %H:%M:%S") > time.strptime(y[:-16],"%a %b %d %Y %H:%M:%S")

In [ ]:
import time
def agregOneProductCartActions(x,y):
    print x
    xTimeString = x[0]
    yTimeString = y[0]
    xTime = time.strptime(xTimeString[:-16], "%a %b %d %Y %H:%M:%S")
    yTime = time.strptime(yTimeString[:-16], "%a %b %d %Y %H:%M:%S")
    returnTime = xTimeString if xTime > yTime else yTimeString
    return (returnTime, 0, x[2]+y[2])

def agregOneProductAccessOfferActions(x,y):
    print x
    xTimeString = x[0]
    yTimeString = y[0]
    xTime = time.strptime(xTimeString[:-16], "%a %b %d %Y %H:%M:%S")
    yTime = time.strptime(yTimeString[:-16], "%a %b %d %Y %H:%M:%S")
    returnTime = xTimeString if xTime > yTime else yTimeString
    return (returnTime, x[1]+y[1])
    

In [ ]:
aggregCartLogs = (cartLogs
                  .map(lambda (a,b,c,d,e,f,g) : ((a,g),(c,d,1)) if f == 'add to cart' else ((a,g),(c,d,-1)))
                  .reduceByKey(agregOneProductCartActions)
                  )
aggregCartLogs.take(5)

## Anomalies pour les actions de type "CART" ##
#### logiquement, après agréggation des 'add' (+1) et 'remove' (-1), l'on ne devrait PAS avoir de valeurs négatives dans le décompte pour le couple (utilisateur, produit). ####
##### Hypothéses : #####
* ##### Certaines valeurs négatives peuvent être expliquées par l'absence des précédentes 'add' actions puisque loggé juste avant la begin date du fichier de logs considéré  ###### 

In [ ]:

print aggregCartLogs.map(lambda x : x[1][2]).countByValue()

In [ ]:
ReadyCartLogs = aggregCartLogs.map(lambda ((k1,k2),(x,y,z)) : ((k1,k2),4.0) if z > 0 else ((k1,k2),2.0) ).cache()

ReadyCartLogs.take(1)

In [ ]:
print accessOfferLogs.count()
aggregAccessOfferLogs = (accessOfferLogs
                  .map(lambda (a,b,c,d,e,f,g) : ((a,g),(c,returnMeANumber(d))))
                  .reduceByKey(agregOneProductAccessOfferActions)
                  )

aggregAccessOfferLogs.take(3)



   ## <span style="color:red">Managing extreme values : assigning a cap value </span>##

In [ ]:
print aggregAccessOfferLogs.filter(lambda x : int(x[1][1])>150000).count() 
logs_in_ms = aggregAccessOfferLogs.filter(lambda x : int(x[1][1])>100000).map(lambda x : int(x[1][1])).collect()

In [ ]:
import numpy as np 
print "Min : %d" %(min(logs_in_ms))
print "Max : %d" %(max(logs_in_ms))


plt.hist(logs_in_ms,range=[90000, 500000])
plt.title("Logs in ms")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

#### Assigning a cap calue of 150.000 for all above extreme values ####

In [ ]:
extremeCappedAggregAccessOfferLogs = aggregAccessOfferLogs.map(lambda x : ((x[0][0],x[0][1]),(x[1][0],100000)) if int(x[1][1])>100000 else x)

## Normalisation ##

In [ ]:
#################### Prends beaucoup de temps ######################
mean = extremeCappedAggregAccessOfferLogs.map(lambda x : int(x[1][1])).mean()
std = extremeCappedAggregAccessOfferLogs.map(lambda x : int(x[1][1])).stdev()
print mean
print std

In [ ]:
mean = 16796.2444516
std = 28172.0146034

In [ ]:

def round_to(n, precision):
    correction = 0.5 if n >= 0 else -0.5
    return int( n/precision+correction ) * precision

def round_to_05(n):
    return round_to(n, 0.05)

print round_to_05(18.75698)

In [ ]:
ReadyAccessOfferLogs = (extremeCappedAggregAccessOfferLogs
                  .map(lambda ((k1,k2),(x,y)) : ((k1,k2), round_to_05((y-mean)/std) ))
                  )
ReadyAccessOfferLogs.take(5)

In [ ]:
print ReadyAccessOfferLogs.take(5)
print ReadyCartLogs.take(5)

In [ ]:
ReadyAllLogs = (ReadyAccessOfferLogs
                .union(ReadyCartLogs)
                .reduceByKey(lambda x,y : x if x > y else y)
                .map(lambda ((x,y),z): (x,y,z))
                .cache()
                )
print ReadyAllLogs.take(1)

In [ ]:

trainingRDD, validationRDD, testRDD = ReadyAllLogs.randomSplit([6, 2, 2], seed=0L) 

# BASELINE #

In [ ]:
def Build_Baseline_RDD(refRDD, testRDD):
        mu = refRDD.map(lambda x : x[2]).mean()
        print mu
        userAverageRatings = (refRDD
                              .map(lambda x : (x[0],(x[2],1)))
                              .reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1]))
                              .map(lambda (x,(y,z)): (x,(y-mu)/z))
                              .collect()
                              )

        productAverageRatings = (refRDD
                              .map(lambda x : (x[1],(x[2],1)))
                              .reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1]))
                              .map(lambda (x,(y,z)): (x,(y-mu)/z))
                              .collect()
                              )
       
        userList = dict(userAverageRatings)
        productList = dict(productAverageRatings)
        
        baselineRDD = testRDD.map(lambda (x,y) : (x,y, mu + userList[x]+productList[y]))
        
        return baselineRDD

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
import math
def auditError(predictedRDD, actualRDD):
    
    # Transform predictedRDD into the tuples of the form ((UserID, MovieID), Rating)
    predictedReformattedRDD = predictedRDD.map(lambda (x,y,z) : ((x,y),z))

    # Transform actualRDD into the tuples of the form ((UserID, MovieID), Rating)
    actualReformattedRDD = actualRDD.map(lambda (x,y,z) : ((x,y),z))
    
    
    
    
def computeError(predictedRDD, actualRDD):
    """ Compute the root mean squared error between predicted and actual
    Args:
        predictedRDD: predicted ratings for each movie and each user where each entry is in the form
                      (UserID, MovieID, Rating)
        actualRDD: actual ratings where each entry is in the form (UserID, MovieID, Rating)
    Returns:
        RSME (float): computed RSME value
    """
    # Transform predictedRDD into the tuples of the form ((UserID, MovieID), Rating)
    predictedReformattedRDD = predictedRDD.map(lambda (x,y,z) : ((x,y),z))

    # Transform actualRDD into the tuples of the form ((UserID, MovieID), Rating)
    actualReformattedRDD = actualRDD.map(lambda (x,y,z) : ((x,y),z))

    # Compute the squared error for each matching entry (i.e., the same (User ID, Movie ID) in each
    # RDD) in the reformatted RDDs using RDD transformtions - do not use collect()
    squaredErrorsRDD = (predictedReformattedRDD
                        .join(actualReformattedRDD)
                        .map(lambda ((x,y),(a,b)):((x,y),math.pow(a-b,2)) )
                       )

    # Compute the total squared error - do not use collect()
    #totalError = squaredErrorsRDD.reduce(lambda (a,b),(c,d) : b+d)
    totalError = squaredErrorsRDD.map(lambda ((x,y),z) : z).sum()

    # Count the number of entries for which you computed the total squared error
    numRatings = squaredErrorsRDD.count()

    # Using the total squared error and the number of entries, compute the RSME
    return math.sqrt(totalError/numRatings)


# sc.parallelize turns a Python list into a Spark RDD.
testPredicted = sc.parallelize([
    (1, 1, 5),
    (1, 2, 3),
    (1, 3, 4),
    (2, 1, 3),
    (2, 2, 2),
    (2, 3, 4)])
testActual = sc.parallelize([
     (1, 2, 3),
     (1, 3, 5),
     (2, 1, 5),
     (2, 2, 1)])
testPredicted2 = sc.parallelize([
     (2, 2, 5),
     (1, 2, 5)])
testError = computeError(testPredicted, testActual)
print 'Error for test dataset (should be 1.22474487139): %s' % testError

testError2 = computeError(testPredicted2, testActual)
print 'Error for test dataset2 (should be 3.16227766017): %s' % testError2

testError3 = computeError(testActual, testActual)
print 'Error for testActual dataset (should be 0.0): %s' % testError3

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
from pyspark.mllib.recommendation import ALS

validationForPredictRDD = validationRDD.map(lambda  (x,y,z): (x,y)).cache()

seed = 5L
iterations = 20
regularizationParameter = 0.1
ranks = [ 16, 20]
errors = [0, 0]
err = 0
tolerance = 0.03

minError = float('inf')
bestRank = -1
bestIteration = -1
for rank in ranks:
    model = ALS.train(trainingRDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularizationParameter)
    predictedRatingsRDD = model.predictAll(validationForPredictRDD)
    error = computeError(predictedRatingsRDD, validationRDD)
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < minError:
        minError = error
        bestRank = rank

print 'The best model was trained with rank %s' % bestRank

In [ ]:
mylist =  validationRDD.map(lambda x : (x[2],1)).reduceByKey(lambda x,y : x+y).collect()

## Confront with baseline figures ##

In [ ]:
baselineValidationSetRDD = Build_Baseline_RDD(ReadyAllLogs, validationForPredictRDD)
print baselineValidationSetRDD.take(20)

In [ ]:
testRMSE = computeError(validationRDD, baselineValidationSetRDD)
print testRMSE

In [ ]:
import csv

with open("toto.csv", 'wb') as f :
    wr = csv.writer(f)
    wr.writerows(mylist)



In [ ]:
#### Lister les profiles utilisateurs par nombre de produits différents consultés ############
UserActionProfiles = (ReadyAllLogs
                     .map(lambda(x,y,z):(x,1))
                     .reduceByKey(lambda x,y : x+y)
                     .map(lambda (x,y):(y,1))
                     .reduceByKey(lambda x,y : x+y)
                     .collect()
                     )

In [ ]:
print UserActionProfiles

In [ ]:
NotationProfiles = (testRDD
                    .map(lambda (x,y,z): (z,1))
                    .reduceByKey(lambda x,y : x+y)
                    .filter(lambda (x,y):x <= -0.1)
                    .takeOrdered(10)

                    )
print NotationProfiles

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
x = np.linspace(0, 3*np.pi, 500)
plt.plot(x, np.sin(x**2))
plt.title('A simple chirp')
plt.show()

In [ ]:
from numpy.random import normal
gaussian_numbers = normal(size=1000)
plt.hist(gaussian_numbers)
plt.title("Gaussian Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:

myModel = ALS.train(trainingRDD, 20, seed=seed, iterations=iterations,
                      lambda_=regularizationParameter)
testForPredictingRDD = testRDD.map(lambda (x,y,z) : (x,y)).cache()
predictedTestRDD = myModel.predictAll(testForPredictingRDD)

testRMSE = computeError(testRDD, predictedTestRDD)

print 'The model had a RMSE on the test set of %s' % testRMSE